In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Finnish language embedding model
model_name = "TurkuNLP/bert-base-finnish-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_sentences(sents: list[str] | str) -> np.ndarray:
    if not isinstance(sents, list):
        sents = [sents]
    inputs = tokenizer(sents, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy()

In [3]:
# Example usage
text = "Koirilla on pentuja."
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1) # sentence embedding (shape: (batch, hidden_size))
embeddings.shape

torch.Size([1, 768])

In [4]:
# Load example text
text = ""
with open('suomi_text.txt', 'r') as f:
    for line in f:
        if line != "\n":
            text += line.replace("\n", "").strip() + " "

In [ ]:
# get sentence embeddings
sents = sent_tokenize(text)
sent_embeddings = np.zeros((len(sents), model.config.hidden_size), dtype="float32")
batch_size = 16
for i in range(0, len(sents), batch_size):
    print(" {:>3}/{}".format(i, len(sents)))
    batch = sents[i : i+batch_size]
    batch_embeds = embed_sentences(batch)
    sent_embeddings[i:i+batch_size, :] = batch_embeds

In [11]:
# Save embeddings
import pickle
id2idx = {}
data = {
    "embeddings": sent_embeddings,
    "id2idx": id2idx,
}
with open("sent_embeddings.pkl", "wb") as f:
    pickle.dump(data, f)

In [12]:
# Load embeddings
with open("sent_embeddings.pkl", "rb") as f:
    data = pickle.load(f)
sent_embeddings = data['embeddings']
id2idx = data['id2idx']

In [13]:
# SIMILARITY
text = "Suomi oli venäjän vallan alla"
query_emb = embed_sentences(text)
sims = cosine_similarity(sent_embeddings, query_emb).squeeze()
sorted_idx = np.flip( np.argsort(sims, axis=0) )[:100].squeeze()
pairs = np.array((sorted_idx, sims[sorted_idx])).T
for idx, sim in pairs[:5]:
    sent = sents[int(idx)]
    print(f"{sim:.4f}  |  {sent}")

0.7724  |  Tämän vuoksi Suomi oli usein Ruotsin ja venäläisten valtioiden (Novgorodin, Moskovan ja keisarillisen Venäjän) välisten sotien taistelutantereena tai sodankäynnin tukialueena.
0.7683  |  Venäjän vallan aika oli Suomen kannalta yksi historian rauhallisimmista kausista (vain Oolannin sota kosketti Suomen aluetta), mutta jonkin verran suomalaisia vapaaehtoisjoukkoja tai upseereita taisteli Venäjän lipun alla Manner-Euroopassa ja Aasiassa.
0.7600  |  Suomi, Baltian maat ja puolet Puolasta kuuluivat Neuvostoliiton etupiiriin.
0.7425  |  Suomen sotahistorian pitkäaikaisena piirteenä oli kuuluminen Ruotsin valtakuntaan keskiajalta vuoteen 1809 asti.
0.7398  |  Vuosina 1809–1917 Suomen suuriruhtinaskunnan sotahistoria oli luonnollisesti sidoksissa Venäjän keisarikuntaan.
